In [ ]:
import numpy as np
from PIL import Image
import taichi as ti
ti.init(arch=ti.cpu, debug=False)
import eikivp
from eikivp.R2.vesselness import vesselness, export_vesselness
from eikivp.utils import cost_function
import matplotlib.pyplot as plt

# Parameters

In [8]:
scales = np.array((0.125, 0.5), dtype=float)
α = 0.5/np.sqrt(2)
γ = 3/4
ε = np.sqrt(0.2)
λ = 1000
p = 2
image_name = "E46_OD_best"
image_file_name = f"data\{image_name}.tif"

params = {
    "scales": scales,
    "α": α,
    "γ": γ,
    "ε": ε,
    "image_name": image_name 
}

# Preprocess Image

In [ ]:
ds = 8
retinal_image = Image.open(image_file_name)
width, height = retinal_image.size
retinal_image_gray_ds = retinal_image.resize((width // ds, height // ds)).convert("L")
retinal_array_unnormalised = np.array(retinal_image_gray_ds).astype(np.float64)
retinal_array = eikivp.R2.utils.align_to_real_axis_scalar_field(eikivp.utils.image_rescale(retinal_array_unnormalised)) # So that +x is in the same direction as +I and +y is in the same direction as +J
eikivp.visualisations.view_image_array(retinal_array);

In [10]:
dim_I, dim_J = retinal_array.shape
Is, Js = np.indices((dim_I, dim_J))
dxy = 1.
x_min = 0.
y_min = 0.
xs, ys = eikivp.R2.utils.coordinate_array_to_real(Is, Js, x_min, y_min, dxy)
x_max = xs[-1, -1]
y_max = ys[-1, -1]

# Compute Vesselness

In [ ]:
V = vesselness(retinal_array, scales, α, γ, ε)
C = cost_function(V, λ, p)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = eikivp.visualisations.plot_image_array(-V, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = eikivp.visualisations.plot_image_array(C, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
export_vesselness(V, params, "storage\\vesselness")